In [1]:
import ee
import geopandas
import numpy as np
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
from shapely.geometry import box
import warnings
from functools import reduce
from math import sin, cos, sqrt, atan2, radians

In [2]:
# Trigger the authentication flow.
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [3]:
gfsad30 = ee.ImageCollection('users/ajsohn/GFSAD30')
gfsad30_lst = gfsad30.toList(gfsad30.size())

In [4]:
for i in range(gfsad30.size().getInfo()):
    img = ee.Image(gfsad30_lst.get(i))
    region = img.getInfo()['id'].split('/')[3]
    if region == 'ne_asia_mos':
        gfsad30_ne_asia_image = img
    print(region)

africa_mos
aus_chn_mos
eu_me_1_mos
eu_me_2_mos
eu_me_3_mos
ne_asia_mos
north_america_mos
se_asia_mos
south_america_mos


In [5]:
center_lon = 129.608463
center_lat = 33.17856
edge_len = 0.1
area_of_interest = ee.Geometry.Rectangle([center_lon-edge_len/2, center_lat-edge_len/2, center_lon+edge_len/2, center_lat+edge_len/2])
aoi_image = gfsad30_ne_asia_image.addBands(ee.Image.pixelLonLat())
aoi_lst = aoi_image.reduceRegion(reducer=ee.Reducer.toList(), geometry=area_of_interest, maxPixels=1e13, scale=30)
b1_series = pd.Series(np.array((ee.Array(aoi_lst.get("b1")).getInfo())), name='b1')
lat_series = pd.Series(np.array((ee.Array(aoi_lst.get("latitude")).getInfo())), name="lat")
lon_series = pd.Series(np.array((ee.Array(aoi_lst.get("longitude")).getInfo())), name="lon")
temp_df = pd.concat([lat_series, lon_series, b1_series], axis=1)

In [6]:
temp_cropland_df = temp_df[temp_df['b1']==2]
random_pixels = []
for lon, lat in zip(temp_cropland_df['lon'].to_list(), temp_cropland_df['lat'].to_list()):
    random_pixels.append((lon, lat))

In [7]:
year=2018
resolution=30
area_of_interest_ee = area_of_interest


def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)

# Reference: https://www.satimagingcorp.com/satellite-sensors/other-satellite-sensors/sentinel-2a/
band_blue = 'B2' #10m
band_green = 'B3' #10m
band_red = "B4"  #10m
band_nir = 'B8'  #10m

def calc_NDVI(img):
    ndvi = ee.Image(img.normalizedDifference([band_nir, band_red])).rename(["ndvi"]).copyProperties(img, img.propertyNames())
    composite = img.addBands(ndvi)
    return composite

# SAVI = ((NIR – Red) / (NIR + Red + L)) x (1 + L)
def calc_SAVI(img):
    """A function to compute Soil Adjusted Vegetation Index."""
    savi =  ee.Image(img.expression(
        '(1 + L) * float(nir - red)/ (nir + red + L)',
        {
            'nir': img.select(band_nir),
            'red': img.select(band_red),
            'L': 0.5
        })).rename(["savi"]).copyProperties(img, img.propertyNames())
    composite = img.addBands(savi)
    return composite

# EVI = 2.5 * ((NIR – Red) / ((NIR) + (C1 * Red) – (C2 * Blue) + L))
#     C1=6, C2=7.5, and L=1
def calc_EVI(img):
    """A function to compute Soil Adjusted Vegetation Index."""
    evi = ee.Image(img.expression(
      '(2.5) * float(nir - red)/ ((nir) + (C1*red) - (C2*blue) + L)',
      {   
          'nir': img.select(band_nir),
          'red': img.select(band_red),
          'blue': img.select(band_blue),
          'L': 0.2,
          'C1': 6,
          'C2': 7.5
      })).rename(["evi"]).copyProperties(img, img.propertyNames())
    composite = img.addBands(evi)
    return composite

def add_landcover(img):
    landcover = ee.Image("USGS/GFSAD1000_V1")
    composite = img.addBands(landcover)
    return composite

def calc_YYYYMM(img):
    return img.set('YYYYMM', img.date().format("YYYYMM"))


In [ ]:
%%time

for month in range(1, 13):
    month_start = datetime.date(year, month, 1)
    month_end = last_day_of_month(datetime.date(year, month, 1))   


    # Create image collection that contains the area of interest
    img_collect = (ee.ImageCollection('COPERNICUS/S2')
                 .filterDate(str(month_start), str(month_end))
                 .filterBounds(area_of_interest_ee)
                    # Remove image that's too small (likely to be partial image)
                    # Size of a full image: 1,276,131,371; size of a partial image: 276,598,191
#                          .filter(ee.Filter.gt('system:asset_size', 800000000))
                 .filterMetadata("CLOUDY_PIXEL_PERCENTAGE","less_than",50)
                  )


    if img_collect.size().getInfo() == 0:
        warnings.warn('No valid image.')
        continue
    print("Total number of images in the collection: ", img_collect.size().getInfo())

    # Extract tile information from each image
    # Note: tiles can overlap a little bit
    unique_tiles = set([item['properties']['MGRS_TILE'] for item in img_collect.getInfo()['features']])
    if len(unique_tiles) > 1:
        print('Number of tiles selected: ', len(unique_tiles))
#             if img_collect_no_partial.size().getInfo() < img_collect.size().getInfo():
#                 warnings.warn('There are partial images in the collection. Proceed with caution.')
#                 print('Number of partial images: ', img_collect.size().getInfo()-img_collect_no_partial.size().getInfo())


    img_collect_calc = img_collect.map(calc_YYYYMM).map(calc_NDVI).map(calc_SAVI).map(calc_EVI).map(add_landcover)

    unique_month = list(set([item['properties']['YYYYMM'] for item in img_collect_calc.getInfo()['features']]))
    unique_month.sort()
    print(unique_month)


    img_calc_month_dict = dict()
    data_dict = dict()
    for month in unique_month:
        img_calc_month_dict[month] = img_collect_calc.filter(ee.Filter.eq('YYYYMM',month)).median()
        img_calc_month2 = img_calc_month_dict[month].addBands(ee.Image.pixelLonLat())
        # EEException: Output of image computation is too large (20 bands for 851968 pixels = 126.8 MiB > 80.0 MiB).
        #     If this is a reduction, try specifying a larger 'tileScale' parameter.
        # EEException: ReduceRegion.AggregationContainer: Valid tileScales are 1 to 16.
        data_dict[month] = pd.DataFrame(columns=["lat", "lon", 'landcover', month+'_NDVI', month+'_SAVI', month+'_EVI'])
        for p in range(len(random_pixels)):
            # `reduceRegion` doesn't like ee.Geometry.MultiPoint as geometry
            data_month_lst = img_calc_month2.reduceRegion(reducer=ee.Reducer.toList(), \
                                                             geometry=ee.Geometry.Point(random_pixels[p]), maxPixels=1e13, scale=resolution)

            # For some reason, ee.Array(data_month_lst.get("...")).getInfo()[0] runs really slow
            pixel_dict = dict()
            pixel_dict['lat'] = ee.Array(data_month_lst.get("latitude")).getInfo()[0]
            pixel_dict['lon'] = ee.Array(data_month_lst.get("longitude")).getInfo()[0]
            try:
                pixel_dict['landcover'] = ee.Array(data_month_lst.get("landcover")).getInfo()[0]
                pixel_dict[month+'_NDVI'] = ee.Array(data_month_lst.get("ndvi")).getInfo()[0]
                pixel_dict[month+'_SAVI'] = ee.Array(data_month_lst.get("savi")).getInfo()[0]
                pixel_dict[month+'_EVI'] = ee.Array(data_month_lst.get("evi")).getInfo()[0]
            except:
                warnings.warn('Missing satellite data.')
                pixel_dict['landcover'] = None
                pixel_dict[month+'_NDVI'] = None
                pixel_dict[month+'_SAVI'] = None
                pixel_dict[month+'_EVI'] = None
            data_dict[month] = data_dict[month].append(pixel_dict, ignore_index=True)

Total number of images in the collection:  3
['201801']


In [8]:
%%time
for month in range(1, 13):
    month_start = datetime.date(year, month, 1)
    month_end = last_day_of_month(datetime.date(year, month, 1))   


    # Create image collection that contains the area of interest
    img_collect = (ee.ImageCollection('COPERNICUS/S2')
                 .filterDate(str(month_start), str(month_end))
                 .filterBounds(area_of_interest_ee)
                    # Remove image that's too small (likely to be partial image)
                    # Size of a full image: 1,276,131,371; size of a partial image: 276,598,191
#                          .filter(ee.Filter.gt('system:asset_size', 800000000))
                 .filterMetadata("CLOUDY_PIXEL_PERCENTAGE","less_than",50)
                  )


    if img_collect.size().getInfo() == 0:
        warnings.warn('No valid image.')
        continue
    print("Total number of images in the collection: ", img_collect.size().getInfo())

    # Extract tile information from each image
    # Note: tiles can overlap a little bit
    unique_tiles = set([item['properties']['MGRS_TILE'] for item in img_collect.getInfo()['features']])
    if len(unique_tiles) > 1:
        print('Number of tiles selected: ', len(unique_tiles))
#             if img_collect_no_partial.size().getInfo() < img_collect.size().getInfo():
#                 warnings.warn('There are partial images in the collection. Proceed with caution.')
#                 print('Number of partial images: ', img_collect.size().getInfo()-img_collect_no_partial.size().getInfo())


    img_collect_calc = img_collect.map(calc_YYYYMM).map(calc_NDVI).map(calc_SAVI).map(calc_EVI).map(add_landcover)

    unique_month = list(set([item['properties']['YYYYMM'] for item in img_collect_calc.getInfo()['features']]))
    unique_month.sort()
    print(unique_month)


    img_calc_month_dict = dict()
    data_dict = dict()
    for month in unique_month:
        img_calc_month_dict[month] = img_collect_calc.filter(ee.Filter.eq('YYYYMM',month)).median()
        img_calc_month2 = img_calc_month_dict[month].addBands(ee.Image.pixelLonLat()
            
        data_month_lst = img_calc_month2.reduceRegion(reducer=ee.Reducer.toList(), \
                                                             geometry=area_of_interest_ee, maxPixels=1e13, scale=resolution)
        try:
            lat_series = pd.Series(np.array((ee.Array(data_month_lst.get("latitude")).getInfo())), name="lat")
            lon_series = pd.Series(np.array((ee.Array(data_month_lst.get("longitude")).getInfo())), name="lon")
            ndvi_series = pd.Series(np.array((ee.Array(data_month_lst.get("ndvi")).getInfo())), name=month+'_NDVI')
            savi_series = pd.Series(np.array((ee.Array(data_month_lst.get("savi")).getInfo())), name=month+'_SAVI')
            evi_series = pd.Series(np.array((ee.Array(data_month_lst.get("evi")).getInfo())), name=month+'_EVI')
            tempppppppp = pd.concat([lat_series, lon_series, ndvi_series, savi_series, evi_series], axis=1)
        except:
            warnings.warn('Missing satellite data.')

Total number of images in the collection:  3
['201801']
Total number of images in the collection:  3
['201802']
Total number of images in the collection:  7
['201803']
Total number of images in the collection:  8
['201804']
Total number of images in the collection:  6
['201805']
Total number of images in the collection:  5
['201806']
Total number of images in the collection:  11
['201807']
Total number of images in the collection:  10
['201808']
Total number of images in the collection:  4
['201809']
Total number of images in the collection:  7
['201810']
Total number of images in the collection:  4
['201811']
Total number of images in the collection:  1
['201812']
CPU times: user 4.44 s, sys: 520 ms, total: 4.96 s
Wall time: 3min 37s
